# Persian Text Classification [DigiMag, Persian News]

The task target is labeling texts in a supervised manner in both existing datasets **DigiMag** and **Persian News**.

In [1]:
!nvidia-smi
!lscpu

Sat Jul 31 12:34:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install hazm==0.7.0
!pip install seqeval==1.2.2
!pip install sentencepiece==0.1.96
!pip install transformers==4.7.0
!pip install clean-text[gpl]==0.4.0

     |████████████████████████████████| 316 kB 7.3 MB/s 
     |████████████████████████████████| 1.4 MB 13.7 MB/s 
     |████████████████████████████████| 233 kB 37.9 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=f4c134a2518cc8dac731df8cafbfa987401d0208583f7316d993d2298409f27b
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154545 sha256=961bac7116156def030d47ffda8d35dfd487b4dccaec62cfaa06c348732a8540
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 43 kB 1.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.

In [3]:
!pip install PyDrive
import os
import IPython.display as ipd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Import required packages
import os
import gc
import re
import hazm
import time
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import AutoModelForSequenceClassification

from cleantext import clean

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print()
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('transformers', transformers.__version__)
print('torch', torch.__version__)
print()

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



numpy 1.19.5
pandas 1.1.5
transformers 4.7.0
torch 1.9.0+cu102

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
class TextClassificationDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Text Classification. """

    def __init__(self, tokenizer, comments, targets, label_list=None, max_len=128):
        self.comments = comments
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label2index = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
        self.index2label = {i: label for label, i in self.label2index.items()}

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, item):
        comment = self.comments[item]
        target = self.label2index[self.targets[item]]
        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt'
        )

        inputs = {
            'comment': comment,
            'targets': torch.tensor(target, dtype=torch.long),
            'original_targets': self.targets[item],
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        return inputs


class TextClassifier:
    def __init__(self, model_name):
        self.normalizer = hazm.Normalizer()
        self.model_name = model_name
        self.config = AutoConfig.from_pretrained(self.model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name)
        self.id2label = self.config.id2label
        self.label2id = self.config.label2id

    def cleaning(self, text):
        def cleanhtml(raw_html):
            clean_pattern = re.compile('<.*?>')
            clean_text = re.sub(clean_pattern, '', raw_html)
            return clean_text

        if type(text) is not str:
            return None

        text = text.strip()

        # regular cleaning
        text = clean(
            text,
            fix_unicode=True,
            to_ascii=False,
            lower=True,
            no_line_breaks=True,
            no_urls=True,
            no_emails=True,
            no_phone_numbers=True,
            no_numbers=False,
            no_digits=False,
            no_currency_symbols=True,
            no_punct=False,
            replace_with_url="",
            replace_with_email="",
            replace_with_phone_number="",
            replace_with_number="",
            replace_with_digit="0",
            replace_with_currency_symbol=""
        )

        # cleaning htmls
        text = cleanhtml(text)

        # normalizing
        text = self.normalizer.normalize(text)

        # removing wierd patterns
        wierd_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u'\U00010000-\U0010ffff'
                                   u"\u200d"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\u3030"
                                   u"\ufe0f"
                                   u"\u2069"
                                   u"\u2066"
                                   # u"\u200c"
                                   u"\u2068"
                                   u"\u2067"
                                   "]+", flags=re.UNICODE)

        text = wierd_pattern.sub(r'', text)

        # removing extra spaces, hashtags
        text = re.sub("#", "", text)
        text = re.sub("\s+", " ", text)
        if text in ['', " "]:
            return None
        return text

    def load_dataset_test_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() == "digimag":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, delimiter="\t")
            # drop label_id because its not consistent with albert model labels!
            data = data[['content', 'label']]

            # cleaning comments
            data = data.dropna(subset=['content'])
            data['content'] = data['content'].apply(self.cleaning)
            data = data.dropna(subset=['content'])

            data['label_id'] = data['label'].apply(lambda t: self.label2id[t])
            x_test, y_test = data['content'].values.tolist(), data['label_id'].values.tolist()
            print(f'test part:\n #content: {len(x_test)}, #labels: {len(y_test)}')
            return x_test, y_test
        if dataset_name.lower() == "persian-news":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, delimiter="\t")
            # drop label_id because its not consistent with albert model labels!
            data = data[['content', 'label']]

            # cleaning comments
            data = data.dropna(subset=['content'])
            data['content'] = data['content'].apply(self.cleaning)
            data = data.dropna(subset=['content'])

            data['label_id'] = data['label'].apply(lambda t: self.label2id[t])
            x_test, y_test = data['content'].values.tolist(), data['label_id'].values.tolist()
            print(f'test part:\n #content: {len(x_test)}, #labels: {len(y_test)}')
            return x_test, y_test

    def text_classification_inference(self, input_text):
        if not self.model or not self.tokenizer or not self.id2label:
            print('Something wrong has been happened!')
            return

        pt_batch = self.tokenizer(
            input_text,
            padding=True,
            truncation=True,
            max_length=self.config.max_position_embeddings,
            return_tensors="pt"
        )

        pt_outputs = self.model(**pt_batch)
        pt_predictions = torch.argmax(F.softmax(pt_outputs.logits, dim=1), dim=1)

        output_predictions = []
        for i, sentence in enumerate(input_text):
            output_predictions.append((sentence, self.id2label.get(pt_predictions[i].item())))
        return output_predictions

    def evaluation(self, input_text, input_labels, device, batch_size=4):
        if not self.model or not self.tokenizer or not self.id2label:
            print('Something wrong has been happened!')
            return

        max_len = self.config.max_position_embeddings
        label_list = list(set(input_labels))
        label_count = {self.id2label[label]: input_labels.count(label) for label in label_list}
        print("label_count:", label_count)
        dataset = TextClassificationDataset(comments=input_text, targets=input_labels, tokenizer=self.tokenizer,
                                            max_len=max_len, label_list=label_list)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

        print("#samples:", len(input_text))
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        total_loss, total_time = 0, 0
        output_predictions = []
        golden_labels, predicted_labels = [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_comments = batch['comment']
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']
            b_token_type_ids = batch['token_type_ids']
            b_targets = batch['targets']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)
            b_token_type_ids = b_token_type_ids.to(device)
            b_targets = b_targets.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model(input_ids=b_input_ids, attention_mask=b_attention_mask,
                                       token_type_ids=b_token_type_ids, labels=b_targets)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')
            # get the loss
            total_loss += b_outputs.loss.item()

            b_original_targets = batch['original_targets']
            golden_labels.extend(b_original_targets.tolist())

            b_predictions = torch.argmax(F.softmax(b_outputs.logits, dim=1), dim=1)
            b_predictions = b_predictions.cpu().detach().numpy().tolist()
            b_predictions = [dataset.index2label[label] for label in b_predictions]
            predicted_labels.extend(b_predictions)

            for i, comment in enumerate(b_comments):
                output_predictions.append((
                    comment,
                    self.id2label[b_original_targets[i].item()],
                    self.id2label[b_predictions[i]]
                ))
                # print(f'output prediction: {i},{comment},{self.id2label[b_original_targets[i].item()]},'
                #       f'{self.id2label[b_predictions[i]]}')

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(data_loader)
        print("average loss:", avg_train_loss)
        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(input_text))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_labels, predicted_labels)))
        print("Test Precision: {}".format(precision_score(golden_labels, predicted_labels, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_labels, predicted_labels, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_labels, predicted_labels, average="weighted")))
        print("Test classification Report:\n{}".format(classification_report(
            golden_labels, predicted_labels, digits=10, target_names=[self.id2label[_] for _ in sorted(label_list)])))
        return output_predictions


In [6]:
model_name='HooshvareLab/bert-fa-base-uncased-clf-persiannews'
tc_model = TextClassifier(model_name)
print(tc_model.config)


BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "persiannews",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\u0627\u062c\u062a\u0645\u0627\u0639\u06cc",
    "1": "\u0627\u0642\u062a\u0635\u0627\u062f\u06cc",
    "2": "\u0628\u06cc\u0646 \u0627\u0644\u0645\u0644\u0644",
    "3": "\u0633\u06cc\u0627\u0633\u06cc",
    "4": "\u0639\u0644\u0645\u06cc \u0641\u0646\u0627\u0648\u0631\u06cc",
    "5": "\u0641\u0631\u0647\u0646\u06af\u06cc \u0647\u0646\u0631\u06cc",
    "6": "\u0648\u0631\u0632\u0634\u06cc",
    "7": "\u067e\u0632\u0634\u06a9\u06cc"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "\u0627\u062c\u062a\u0645\u0627\u0639\u06cc": 0,
    "\u0627\u0642\u062a\u0635\u0627\u062f\u06cc": 1,
    "\u0628\u06cc\u0646 \u0627\u0644\u0645\u0644\u0644": 2,
    "\u0633\u06cc\u0627\u0633

In [7]:
print(tc_model.id2label)
print(tc_model.label2id)

{0: 'اجتماعی', 1: 'اقتصادی', 2: 'بین الملل', 3: 'سیاسی', 4: 'علمی فناوری', 5: 'فرهنگی هنری', 6: 'ورزشی', 7: 'پزشکی'}
{'اجتماعی': 0, 'اقتصادی': 1, 'بین الملل': 2, 'سیاسی': 3, 'علمی فناوری': 4, 'فرهنگی هنری': 5, 'ورزشی': 6, 'پزشکی': 7}


## Persian News
A dataset of various news articles scraped from different online news agencies' websites. The total number of articles is 16,438, spread over eight different classes.

|          Label         | # | 
|:------------------------:|:-----------:|
|  Social  |      2170    |
|  Economic  |      1564    |
|  International |      1975      |
|  Political |      2269      |
|  Science Technology |      2436      |
|  Cultural Art |      2558      |
|  Sport |      1381      |
|  Medical |      2085      |

Download You can download the dataset from [here](https://drive.google.com/uc?id=1B6xotfXCcW9xS1mYSBQos7OCg0ratzKC).	

In [8]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id': '1B6xotfXCcW9xS1mYSBQos7OCg0ratzKC'})
download.GetContentFile('persian_news.zip')
!ls

adc.json  persian_news.zip  sample_data


In [9]:
!unzip persian_news.zip
!ls
!ls persian_news

Archive:  persian_news.zip
   creating: persian_news/
  inflating: persian_news/dev.csv    
  inflating: persian_news/train.csv  
  inflating: persian_news/test.csv   
adc.json  persian_news	persian_news.zip  sample_data
dev.csv  test.csv  train.csv


In [10]:
test_comments, test_labels = tc_model.load_dataset_test_file(dataset_name="persian-news", dataset_file="./persian_news/test.csv")
print(test_comments[:5])
print(test_labels[:5])
print(len(test_comments))
print(len(test_labels))

test part:
 #content: 1644, #labels: 1644
['حسن جوهرچی بازیگر سینما و تلویزیون ایران در گفتگو با خبرنگار حوزه سینما گروه فرهنگی باشگاه خبرنگاران جوان؛ در خصوص علت کم کاری\u200cاش در چند سال اخیر گفت: با در نظر گرفتن نبود بودجه کافی که در پی آن تولید کم خواهد شد اکثر بازیگران کم کار می\u200cشوند امیدوارم وضعیت بودجه رو به بهبود رود و تولیدات مثل قدیم افزایش یابد تا اینکه حرکت جدیدی اتفاق بیفتد و ما دوباره به طور دائم سرکار باشیم. وی در خصوص حال و هوای این روزهای سینما ایران بیان کرد: به نظر می\u200cرسد که سینما کم\u200cکم در حال تعطیل شدن است، یعنی آثار سینمایی ما مخاطب را جذب نمی\u200cکند، سالن\u200cها خالی و فیلمسازان خوب با کم کاری و یا اصلا فعالیت نمی\u200cکنند که این جای تاسف دارد، امیدوارم مسئولان سینمایی فکری به حال این موضوع به خصوص در بخش تولید و فیلم\u200cنامه داشته باشند تا اینکه سینما به روزهای درخشان خود بازگردد. وی ادامه داد: بحث فیلم\u200cنامه اولین موضوع در تولید و ساختار یک فیلم است و سینمای ما از قدیم\u200cالایام با این جریان مشکل داشته در صورتی که اواخر دهه شصت و اوای

In [11]:
tc_model.text_classification_inference(test_comments[:5])

[('حسن جوهرچی بازیگر سینما و تلویزیون ایران در گفتگو با خبرنگار حوزه سینما گروه فرهنگی باشگاه خبرنگاران جوان؛ در خصوص علت کم کاری\u200cاش در چند سال اخیر گفت: با در نظر گرفتن نبود بودجه کافی که در پی آن تولید کم خواهد شد اکثر بازیگران کم کار می\u200cشوند امیدوارم وضعیت بودجه رو به بهبود رود و تولیدات مثل قدیم افزایش یابد تا اینکه حرکت جدیدی اتفاق بیفتد و ما دوباره به طور دائم سرکار باشیم. وی در خصوص حال و هوای این روزهای سینما ایران بیان کرد: به نظر می\u200cرسد که سینما کم\u200cکم در حال تعطیل شدن است، یعنی آثار سینمایی ما مخاطب را جذب نمی\u200cکند، سالن\u200cها خالی و فیلمسازان خوب با کم کاری و یا اصلا فعالیت نمی\u200cکنند که این جای تاسف دارد، امیدوارم مسئولان سینمایی فکری به حال این موضوع به خصوص در بخش تولید و فیلم\u200cنامه داشته باشند تا اینکه سینما به روزهای درخشان خود بازگردد. وی ادامه داد: بحث فیلم\u200cنامه اولین موضوع در تولید و ساختار یک فیلم است و سینمای ما از قدیم\u200cالایام با این جریان مشکل داشته در صورتی که اواخر دهه شصت و اوایل دهه هفتاد آثار خوبی در سینما ساخته می\u

In [12]:
!nvidia-smi
!lscpu

Sat Jul 31 12:36:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
evaluation_output = tc_model.evaluation(test_comments, test_labels, device, batch_size=128)

label_count: {'اجتماعی': 217, 'اقتصادی': 156, 'بین الملل': 197, 'سیاسی': 227, 'علمی فناوری': 244, 'فرهنگی هنری': 256, 'ورزشی': 138, 'پزشکی': 209}
#samples: 1644
#batch: 13
Start to evaluate test data ...
inference time for step 0: 0.13143265300001872
inference time for step 1: 0.008464570000001004
inference time for step 2: 0.008244885000010527
inference time for step 3: 0.008315233000018907
inference time for step 4: 0.008163499999994883
inference time for step 5: 0.008325052999992977
inference time for step 6: 0.008229394000011325
inference time for step 7: 0.008329829000018663
inference time for step 8: 0.008378284000002623
inference time for step 9: 0.00863049399998772
inference time for step 10: 0.008295795999998745
inference time for step 11: 0.008581225000000359
inference time for step 12: 0.008673652999988235
average loss: 0.04895865496543523
total inference time: 0.23206456900004468
total inference time / #samples: 0.00014115849695866466
Test Accuracy: 0.9872262773722628
Test 

In [14]:
for comment, true_label, predicted_label in evaluation_output[:25]:
  print('{}\t{}\t{}'.format(comment, true_label, predicted_label))

حسن جوهرچی بازیگر سینما و تلویزیون ایران در گفتگو با خبرنگار حوزه سینما گروه فرهنگی باشگاه خبرنگاران جوان؛ در خصوص علت کم کاری‌اش در چند سال اخیر گفت: با در نظر گرفتن نبود بودجه کافی که در پی آن تولید کم خواهد شد اکثر بازیگران کم کار می‌شوند امیدوارم وضعیت بودجه رو به بهبود رود و تولیدات مثل قدیم افزایش یابد تا اینکه حرکت جدیدی اتفاق بیفتد و ما دوباره به طور دائم سرکار باشیم. وی در خصوص حال و هوای این روزهای سینما ایران بیان کرد: به نظر می‌رسد که سینما کم‌کم در حال تعطیل شدن است، یعنی آثار سینمایی ما مخاطب را جذب نمی‌کند، سالن‌ها خالی و فیلمسازان خوب با کم کاری و یا اصلا فعالیت نمی‌کنند که این جای تاسف دارد، امیدوارم مسئولان سینمایی فکری به حال این موضوع به خصوص در بخش تولید و فیلم‌نامه داشته باشند تا اینکه سینما به روزهای درخشان خود بازگردد. وی ادامه داد: بحث فیلم‌نامه اولین موضوع در تولید و ساختار یک فیلم است و سینمای ما از قدیم‌الایام با این جریان مشکل داشته در صورتی که اواخر دهه شصت و اوایل دهه هفتاد آثار خوبی در سینما ساخته می‌شد اما متاسفانه این حرکت ادامه پیدا نکرد. جوهرچی بیان 

In [15]:
output_file_name = "text_classification_persian-news_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for comment, true_label, predicted_label in evaluation_output:
    output_file.write('{}\t{}\t{}\n'.format(comment, true_label, predicted_label))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()